In [1]:
import requests
import json
import pandas as pd
import time
import telegram

import enpyong as ep

headers= {
'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
      } 



In [3]:
def crawl(url):
  res = requests.get(url, headers=headers)

  #크롤링한 데이터를 Json형태로 변환
  data = json.loads(res.text)
  items = data['result']

  #변환된 Json데이터를 DataFrame화
  df = pd.DataFrame.from_dict(items)
  df = df.rename(columns={'hscpNm':'단지명','dealPrcMin':'매매최저가','dealPrcMax':'매매최고가',
                        'leasePrcMin':'전세최저가','leasePrcMax':'전세최고가','useAprvYmd':'승인날짜'})

  #불필요한 문자들 정규식으로 삭제
  df["매매최저가"].replace("[a-z<='_>/]", "", regex=True, inplace = True) 
  df["매매최고가"].replace("[a-z<='_>/]", "", regex=True, inplace = True)
  df["전세최저가"].replace("[a-z<='_>/]", "", regex=True, inplace = True) 
  df["전세최고가"].replace("[a-z<='_>/]", "", regex=True, inplace = True)

  #원하는 DataFrame 컬럼을 뽑아온뒤 지역엑셀 파일로 저장
  P_list = df[['단지명','전세최저가','전세최고가','승인날짜','매매최저가','매매최고가']]

  return P_list


In [24]:
#은평구 수색동
su = crawl(ep.url_susaek)
su.to_excel('./Legion/수색.xlsx')

#은평구 갈현동
gal1 = crawl(ep.url_galhyun1)
gal2 = crawl(ep.url_galhyun2)
gal3 = crawl(ep.url_galhyun3)

#데이터프레임3개를 합침
galdf = pd.concat([gal1, gal2, gal3], ignore_index = True)
galdf.to_excel('./Legion/갈현.xlsx')

#은평구 구산동
gu1 = crawl(ep.url_gusan1)
gu2 = crawl(ep.url_gusan2)
gu3 = crawl(ep.url_gusan3)

gudf = pd.concat([gu1, gu2, gu3], ignore_index = True)
gudf.to_excel('./Legion/구산.xlsx')

#은평구 녹번동
nok = crawl(ep.url_nokbun)
nok.to_excel('./Legion/녹번.xlsx')

#은평구 대조동
dae1 = crawl(ep.url_daejo1)
dae2 = crawl(ep.url_daejo2)
dae3 = crawl(ep.url_daejo3)

daedf = pd.concat([dae1, dae2, dae3], ignore_index= True)
daedf.to_excel('./Legion/대조.xlsx')

#은평구 불광동
bul_list = []

bul1 = crawl(ep.url_bulkwang1)
bul2 = crawl(ep.url_bulkwang2)
bul3 = crawl(ep.url_bulkwang3)
bul4 = crawl(ep.url_bulkwang4)
bul5 = crawl(ep.url_bulkwang5)

buldf = pd.concat([bul1, bul2, bul3, bul4, bul5], ignore_index= True)
buldf.to_excel('./Legion/불광.xlsx')



In [14]:
telegram_config = {}
# config 파일 읽기
with open('./Mod/telegram_config', 'r') as f:
    # 모든 줄 읽어오기
    configs = f.readlines()
    # 한 줄씩 확인해서
    for config in configs:
        # 줄바꿈기호 제거(\n) 후 =로 문자열 분리
        # key, value로 언패킹 (2개 나올것이 확실하기 때문에)
        key, value = config.rstrip().split('=')
        # config 딕셔너리에 키-값 추가
        telegram_config[key] = value

token = telegram_config['token']


In [15]:
bot = telegram.Bot(token)

In [16]:
updates = bot.get_updates()

In [17]:
chat_id = updates[-1].message.chat.id

In [18]:
last_id = updates[-1].update_id

In [22]:
# 서버에서 텔레그램 메시지 확인, 응답 보내기
bot.send_message(chat_id, '은평구 아파트 단지 시세조회 시작!\n원하는 동을 입력하세요\n(갈현, 구산, 녹번, 대조, 불광, 수색)')
while True:
    
    try:
        # 신규 메시지가 없을 경우 에러가 발생 
        # list index out of range
        # 따라서, try - except 문으로 묶어줌
        
        new_message = bot.get_updates(offset=last_id)[-1]

        if new_message.message.text == '갈현':
            # 관련 메시지 발송
            bot.send_document(chat_id, open('./Legion/갈현.xlsx', 'rb'))
        
        elif new_message.message.text == '구산':
            bot.send_document(chat_id, open('./Legion/구산.xlsx', 'rb'))

        elif new_message.message.text == '녹번':
            bot.send_document(chat_id, open('./Legion/녹번.xlsx', 'rb'))

        elif new_message.message.text == '대조':
            bot.send_document(chat_id, open('./Legion/대조.xlsx', 'rb'))

        elif new_message.message.text == '불광':
            bot.send_document(chat_id, open('./Legion/불광.xlsx', 'rb'))    

        elif new_message.message.text == '수색':
            bot.send_document(chat_id, open('./Legion/수색.xlsx', 'rb'))

           


        # offset 값 최신화 (update_id) + 1 해줘서 그 다음부터 메시지부터 확인하도록
        last_id = new_message.update_id + 1
    except:
        pass

    # 텔레그램 서버 부하 줄이기 위해 3초마다 확인
    time.sleep(3)





KeyboardInterrupt: 